<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.981 · TFM · Aula 1</p>
<p style="margin: 0; text-align:right;">2024-2 · Màster universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'informàtica, multimèdia i telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>
<div style="text-align: center; margin-top: 40px;">
    <h1>Aplicación para ayuda a la conducción con deep learning</h1>
</div>

### ARCHIVO 9: Extracción imágenes frenos + semáforos

Crea un dataset

Código que ejecuta en vídeos e imágenes del modelos entrenado para extraer bbox de luces posteriores de vehículos y semáforos. 

Código ejecutado en plataforma Visual Studio Code.

In [1]:
# Llibreries

from ultralytics import YOLO
import cv2
import os
import numpy as np
from pathlib import Path

In [2]:
# Models

model_dve = YOLO("Models/Detectvehicles.pt")    # Model que detecta cotxes i motos
model_seg = YOLO("Models/CarSegment.pt")        # Model que segmenta cotxes
model_dss = YOLO("Models/Detect_Signs.pt")      # Model que detecta semàfors i STOPS


In [ ]:
# Directoris

input_dir = Path("Imatges")                     # Ubicació de les imatges
output_dir_lights = Path("Llums_Cotxe")         # Ubicació final de pilots posteriors
output_dir_tralig = Path("Semàforss")            # Ubicació final de semàfors

### Extracción de pilotos posteriores

In [10]:
# Selecció índex de pilots posteriors
target_classes = [4, 5, 7] 

# Bucle
for img_path in list(input_dir.glob("*.jpg")) + list(input_dir.glob("*.jpeg")):
    frame = cv2.imread(str(img_path))

    # Detecció
    results_det = model_dve(frame)[0]
    detections = results_det.boxes

    for box in detections:
        cls = int(box.cls[0])
        label = model_dve.names[cls]

        # Coordenades
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        x1, y1 = max(x1, 0), max(y1, 0)
        x2, y2 = min(x2, frame.shape[1]), min(y2, frame.shape[0])

        # SEGMENTACIÓ
        roi = frame[y1:y2, x1:x2]                               # Regió de la imatge
        results_seg = model_seg(roi, verbose=False)[0]

        if results_seg.masks is not None:                       # Si segmenta, es mostren els resultats
            # Filtre pilots posteriors
            for i, mask in enumerate(results_seg.masks.data):
                cls_id = int(results_seg.boxes.cls[i])          # Extreu l'índex
                if cls_id in target_classes:
                    mask_np = mask.cpu().numpy().astype("uint8") * 255
                    mask_resized = cv2.resize(mask_np, (x2 - x1, y2 - y1))

                    ys, xs = np.where(mask_resized > 0)
                    if len(xs) == 0 or len(ys) == 0:
                        continue

                    px1, py1, px2, py2 = min(xs), min(ys), max(xs), max(ys)
                    crop = roi[py1:py2, px1:px2]

                    out_path = output_dir_lights / f"{img_path.stem}_{cls_id}.jpg" # Nom arxiu
                    cv2.imwrite(str(out_path), crop)                          # Guarda

0: 480x640 5 cars, 46.7ms
Speed: 5.9ms preprocess, 46.7ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 10 cars, 14.6ms
Speed: 3.2ms preprocess, 14.6ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 cars, 13.8ms
Speed: 3.5ms preprocess, 13.8ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 14 cars, 14.0ms
Speed: 3.3ms preprocess, 14.0ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 cars, 17.1ms
Speed: 5.4ms preprocess, 17.1ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 12 cars, 15.3ms
Speed: 3.5ms preprocess, 15.3ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 cars, 1 motorcycle, 14.6ms
Speed: 3.4ms preprocess, 14.6ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 13 cars, 16.1ms
Speed: 3.7ms preprocess, 16.1ms inference, 2.6ms postprocess per image at shape (1, 3, 4

### Extracción de semáforos

In [14]:
# Selecció índex de semàfors
target_classes = [0] 

# Bucle
for img_path in list(input_dir.glob("*.jpg")) + list(input_dir.glob("*.jpeg")):
    frame = cv2.imread(str(img_path))

    # Detecció
    results_det = model_dss(frame)[0]
    detections = results_det.boxes

    for box in detections:
        cls = int(box.cls[0])
        label = model_dve.names[cls]
        if cls not in target_classes:    # Filtre
            continue

        # Coordenades
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        x1, y1 = max(x1, 0), max(y1, 0)
        x2, y2 = min(x2, frame.shape[1]), min(y2, frame.shape[0])

        # Extracció
        crop = frame[y1:y2, x1:x2].copy()
        out_path = output_dir_tralig / f"{img_path.stem}_tl.jpg"
        cv2.imwrite(str(out_path), crop)


0: 480x640 1 traffic light, 1 stop sign, 130.0ms
Speed: 29.6ms preprocess, 130.0ms inference, 6.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 traffic lights, 45.4ms
Speed: 5.9ms preprocess, 45.4ms inference, 4.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 traffic light, 24.6ms
Speed: 4.1ms preprocess, 24.6ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 traffic lights, 23.8ms
Speed: 3.2ms preprocess, 23.8ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 16.7ms
Speed: 3.3ms preprocess, 16.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 traffic lights, 16.1ms
Speed: 3.4ms preprocess, 16.1ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 traffic lights, 14.2ms
Speed: 3.2ms preprocess, 14.2ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 traffic lights, 16.7ms
Speed: 4.5ms